In [92]:
import re
import jieba
import nltk
nltk.download('words')
nltk.download('punkt')
eng_words = set(nltk.corpus.words.words())
from nltk.tokenize import sent_tokenize
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
# from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import PCA
from urllib.parse import unquote
# from gensim.test.utils import common_texts, get_tmpfile
# from gensim.models import Word2Vec
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [96]:
search_keyword = 'seo 公司'
num = 50

####設定停用詞
# cn_stopwords = pd.read_csv('cn_stopwords.csv')['$'].values.tolist()
# 更完整的停用詞庫: https://github.com/tomlinNTUB/Machine-Learning/blob/master/data/%E5%81%9C%E7%94%A8%E8%A9%9E-%E7%B9%81%E9%AB%94%E4%B8%AD%E6%96%87.txt
file = open('cn_stopwords.txt', 'r', encoding='utf-8')
try:
    content = file.read()
    cn_stopwords = content.split('\n')
finally:
    file.close()
    
exclude_cn_marks = ['〈', '〉','＜','＞','《','》','｛','｝','﹛','﹜','［','］','「','」','『','』','【','】','〔','〕','。','，','、','；','：','»','｜','！','？','–','…','©']
for char in exclude_cn_marks:
    cn_stopwords.append(char)

file = open('eng_stopwords.txt', 'r')
try:
    content = file.read()
    eng_stopwords = content.split(',')
finally:
    file.close()

# https://blog.csdn.net/linshenwei1995/article/details/78987444
exclude_marks = ['/','\\','>','<','»','->','!','@','#','$','^','&','*','(',')','-','=','+','{','}','[',']','|',';',':','.','..','...',',','?','~','`',"'",'\n','\r\n','\xa0']
for char in exclude_marks:
    eng_stopwords.append(char)

all_stopwords = list()
for char in cn_stopwords:
    all_stopwords.append(char)
for char in eng_stopwords:
    all_stopwords.append(char)
    
all_marks = exclude_cn_marks + exclude_marks

####啟動 tf-idf 計算函式
# vectorizer = TfidfVectorizer(analyzer='word', stop_words=all_stopwords)#, max_df=1.0, min_df=0.2)


In [97]:
def sub_marks(x):
    text_list = list(x)
    new_list = list()
    for char in text_list:
        if char not in all_marks:
            new_list.append(char)
        else:
            new_list.append(' ')
    return ''.join(new_list)

page_data = pd.read_csv('SERP_{}_num_{}.csv'.format(search_keyword, num))
df = pd.read_csv('{}_{}_articles_features.csv'.format(search_keyword, num))
df = pd.concat([df.iloc[:, np.r_[0, 2:7]], page_data.iloc[:, :2]], axis=1)
# df['link'] = df['link'].apply(lambda x: unquote(x).lower())
# df['article_content'] = df['article_content'].apply(lambda x: sub_marks(x.lower()))
# df['title'] = df['title'].apply(lambda x: sub_marks(x.lower()))
# df['desc'] = df['desc'].apply(lambda x: sub_marks(x.lower()))
# df['web_content_feature'] = df['article_content']+' '+(df['link']+' ')*30+' '+(df['title']+' ')*20+' '+(df['desc']+' ')*2

print(df.shape)
df.head(1)


(50, 8)


,link,article_keywords,article_content,article_published_time,article_modified_time,article_schema,title,desc
0,https://www.da-vinci.com.tw/tw/blog/seo-cost,NaN,搜尋 搜尋 f TW TW CN 首頁 關於我們 公司介紹 專業團隊 服務項目 網頁設計 網...,NaN,NaN,NaN,SEO收費行情？該怎麼選擇SEO公司?-SEO優化-部落格-達文西 ...,2020年3月17日 - 大家都在討論SEO關鍵字排名，市場報價非常紊亂，比網頁設計更沒有報...


#### Split to sentence 
- delimiter: 。, ？, ！, ?, !


In [98]:
df['article_content'] = df['article_content'].apply(lambda x: re.split('[。？！?!]', str(x)))  # https://stackoverflow.com/questions/4998629/split-string-with-multiple-delimiters-in-python

sentences = list()

# for article in df['article_content'].values:
#     sentences.append(sent_tokenize(article))

for item_list in df['article_content'].values:
    for item in item_list:
        if (item != '') and (item != ' '):
            sentences.append(item)

np.array(sentences).shape
sentences[0]


(1798,)

'搜尋 搜尋 f TW TW CN 首頁 關於我們 公司介紹 專業團隊 服務項目 網頁設計 網頁設計服務 客製網頁設計 套版網頁設計 Da-系列服務 Da-Video 影音網 Da-Shop 購物網 主機服務 網站代管 其他服務 SEO全網站優化 EDM主動行銷 商業攝影 設計作品 常見問答 網頁設計 SEO網站優化 網站主機代管 Da-Video 影音網 Da-Shop 購物網 部落格 網頁設計 網路行銷 SEO優化 工商服務 聯絡我們 BLOG 首頁 部落格 SEO收費行情'

#### clean each sentence


In [111]:
test_sentences = sentences[:62]
test_sentences = [sub_marks(x.lower()) for x in test_sentences]

def jieba_cut_in_sentence(x):
    _list = list()
    for char in jieba.cut(x, cut_all=False):
        if ((x.strip() != '') and (x.strip() not in all_stopwords) and (any(char.isdigit() for char in x.strip()) == False)):
            _list.append(char)
    
    return _list
            
test_sentences = [' '.join([x for x in jieba_cut_in_sentence(x) if x != ' ']) for x in test_sentences]
test_sentences[0]


'搜尋 搜尋 f tw tw cn 首頁 關於 我們 公司 介紹 專業 團隊 服務 項目 網頁 設計 網頁 設計 服務 客 製 網 頁 設 計 套版 網頁 設計 da 系列 服務 da video 影音 網 da shop 購物網 主機 服務 網站 代管 其他 服務 seo 全網 站 優化 edm 主動 行銷 商業 攝影 設計 作品 常見 問答 網頁 設計 seo 網站 優化 網站 主機 代管 da video 影音 網 da shop 購物網 部落 格 網頁 設計 網路 行銷 seo 優化 工商 服務 聯絡 我們 blog 首頁 部落 格 seo 收費 行情'

### Word Embedding: GloVe

In [114]:
word_embeddings = {}
f = open('glove.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    vs = line.split()
    word = vs[0]
    coefs = np.asarray(vs[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

len(word_embeddings)
    

400000

#### Create vectors from cleaned sentences

In [112]:
sentence_vectors = list()

for sent in test_sentences:
    if len(sent) != 0:
        


SyntaxError: invalid syntax (<ipython-input-112-25e181702741>, line 3)

In [1]:
word_embeddings.get('seo', np.zeros((100,)))

### 現在問題：GloVe只有英文單詞的embedding

NameError: name 'word_embeddings' is not defined